# **Objectives**
in this project, you will:
* load the data as a pandas dataframe
* cleann the data, taking care of the blank entries
* run exploratory data analysis(EDA) and identify the attributes that most affect the charges
* develop single variable and multi-variable Linear Regression Models for predictinf the charges
* Use Ridge regression to refine the performance of Linear regression models

In [1]:
#import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split